# Populate Airtable by scraping Builtin

This pulls several pages worth of companies from a Builtin site (like builtinseattle.com) and inserts the new ones into Airtable in the pending state.


In [10]:
from core import init

# Initialize the HTTP request cache
init()


In [11]:
from data_sources.builtin import scrape

seattle_companies = scrape("seattle", 5)

Fetching company list from https://www.builtinseattle.com/companies
Fetching company details for 2K from https://www.builtinseattle.com/company/2k
Fetching company details for Liberty Mutual Insurance from https://www.builtinseattle.com/company/liberty-mutual-insurance
Fetching company details for PEAK6 from https://www.builtinseattle.com/company/peak6
Fetching company details for Atlassian from https://www.builtinseattle.com/company/atlassian
Fetching company details for Monte Carlo from https://www.builtinseattle.com/company/monte-carlo
Fetching company details for Sprout Social from https://www.builtinseattle.com/company/sprout-social
Fetching company details for Ontra from https://www.builtinseattle.com/company/ontra
Fetching company details for Block from https://www.builtinseattle.com/company/block
Fetching company details for TBD from https://www.builtinseattle.com/company/tbd
Fetching company details for Square from https://www.builtinseattle.com/company/square
Fetching company

In [12]:
import pandas as pd
pd.options.display.max_rows = 100

seattle_companies

,description,link,industry,size,location,employees,founded,website,hiring_by_category
name,,,,,,,,,
2K,"2K is headquartered in Novato, California and ...",/company/2k,Gaming • Information Technology • Mobile • Sof...,"3,505 Employees","Kirkland, WA","3,505 Total Employees",Year Founded: 2005,https://2k.com/en-us/jobs/,"{'Design + UX': {'num_roles': '1', 'link': '/j..."
Liberty Mutual Insurance,Liberty Mutual Insurance exists to help people...,/company/liberty-mutual-insurance,Artificial Intelligence • Insurance • Marketin...,"45,384 Employees","Seattle, WA","45,384 Total Employees",Year Founded: 1912,https://jobs.libertymutualgroup.com/digital-te...,"{'Developer + Engineer': {'num_roles': '7', 'l..."
PEAK6,PEAK6 is not your typical holding company. We'...,/company/peak6,Fintech,"1,900 Employees","Seattle, WA","1,900 Total Employees",Year Founded: 1997,http://www.peak6.com/,"{'Finance': {'num_roles': '1', 'link': '/jobs/..."
Atlassian,Atlassian builds software that empowers everyo...,/company/atlassian,Cloud • Information Technology • Productivity ...,"11,000 Employees",None,"11,000 Total Employees",Year Founded: 2012,https://www.atlassian.com/,"{'Developer + Engineer': {'num_roles': '48', '..."
Monte Carlo,As businesses increasingly rely on data to pow...,/company/monte-carlo,Big Data • Cloud • Software • Big Data Analytics,165 Employees,"Bellevue, WA",165 Total Employees,Year Founded: 2019,http://www.montecarlodata.com,"{'Developer + Engineer': {'num_roles': '3', 'l..."
Sprout Social,Sprout Social offers deep social media listeni...,/company/sprout-social,Marketing Tech • Social Media • Software,"1,485 Employees","Seattle, WA","1,485 Total Employees",Year Founded: 2010,https://sproutsocial.com/?utm_source=builtinch...,"{'Sales': {'num_roles': '2', 'link': '/jobs/sa..."
Ontra,Ontra is the global leader in Contract Automat...,/company/ontra,Artificial Intelligence • Legal Tech • Profess...,350 Employees,"Seattle, WA",350 Total Employees,Year Founded: 2014,https://www.ontra.ai/,"{'Data + Analytics': {'num_roles': '3', 'link'..."
Block,"Block, Inc. (NYSE: SQ) is a global technology ...",/company/block,Blockchain • eCommerce • Fintech • Payments • ...,"12,000 Employees","Seattle, WA","12,000 Total Employees",Year Founded: 2009,https://block.xyz/,"{'Developer + Engineer': {'num_roles': '7', 'l..."
TBD,TBD is Block’s business unit that’s building a...,/company/tbd,Blockchain • Fintech • Financial Services • Cr...,190 Employees,None,190 Total Employees,Year Founded: 2021,https://tbd.website/,{}


# Insert into Airtable!

In [13]:
# Step 1: Get the existing Airtable data so that we can deduplicate
import airtable

airtable_df = airtable.load_into_pandas(status=None)
airtable_df

,id,createdTime,fields.Name,fields.Website,fields.Status,fields.Key Product Name,domain
0,rec8leUf8FyDrIa8x,2024-09-30T20:37:50.000Z,Akili Interactive,https://www.akiliinteractive.com/,Approved,NaN,akiliinteractive.com
1,recC1pqGYRxzcmoFs,2024-09-30T20:37:09.000Z,Synthesize Bio,https://www.synthesize.bio/,Approved,NaN,synthesize.bio
2,recE8NhvREVz0cNIa,2024-09-30T20:34:11.000Z,98point6,98point6.com,Approved,NaN,98point6.com
3,recFDpfbo41ICr8O8,2024-09-30T20:35:54.000Z,Kevala,kevala.care,Approved,NaN,kevala.care
4,recMjV9bqTNfmN1yO,2024-10-01T21:32:27.000Z,Rad AI,radai.com,Pending,Impressions,radai.com
5,recSxlXd9ZOPUr8dL,2024-09-30T20:34:55.000Z,Imagine Pediatrics,imaginepediatrics.org,Approved,NaN,imaginepediatrics.org
6,recU56sRD2MLNN9lz,2024-09-30T20:35:03.000Z,Pomelo Care,pomelocare.com,Approved,NaN,pomelocare.com
7,recaVe1KAvdyjHkR6,2024-09-30T20:35:22.000Z,Singularity 6,singularity6.com,Approved,Palia,singularity6.com
8,recwZi2dVuPxfaDnd,2024-09-30T20:36:13.000Z,Rippling,rippling.com,Approved,NaN,rippling.com


In [14]:
airtable_ready = seattle_companies.copy().reset_index()

# Drop ones that are already in Airtable
# TODO: Deduplicate by the domain rather than the company name to better handle companies with official names like "Hiya, Inc." but which should be searched for as "Hiya"
airtable_ready = airtable_ready[~airtable_ready["name"].isin(airtable_df["fields.Name"])]

# Adjust columns
airtable_ready["extra_json"] = airtable_ready.apply(lambda x: x.to_json(), axis=1)
airtable_ready.rename(columns={"name": "Name", "website": "Website"}, inplace=True)

# Format for Airtable
airtable_batch_insert = airtable_ready[['Name', 'Website', 'extra_json']].to_dict(orient="records")
airtable_batch_insert

[{'Name': '2K',
  'Website': 'https://2k.com/en-us/jobs/',
  'extra_json': '{"name":"2K","description":"2K is headquartered in Novato, California and is a wholly owned label of Take-Two Interactive Software, Inc. (NASDAQ: TTWO). Founded in 2005, 2K Games is a global video game company, publishing titles developed by some of the most influential game development studios in the world. Our studios responsible for developing 2K\\u2019s portfolio of world-class games across multiple platforms, include Visual Concepts, Firaxis, Hangar 13, CatDaddy, Cloud Chamber, 31st Union, and HB Studios. Our portfolio of titles is expanding due to our global strategic plan, building and acquiring exciting studios whose content continues to inspire all of us! 2K publishes titles in today\\u2019s most popular gaming genres, including sports, shooters, action, role-playing, strategy, casual, and family entertainment.\\r\\n\\r\\nOur team of engineers, marketers, artists, writers, data scientists, producers, t

In [15]:
if airtable_batch_insert:
    airtable.company_table.batch_create(airtable_batch_insert)
    print(f"Inserted {len(airtable_batch_insert)} new records into Airtable")